One our trading strategies requires forecasts of volatility (= std dev of returns, returns typically mean log returns i.e. log(P(t)) - log(P(t-1)). 
So we are always looking for an edge here. I have attached a recently published paper on predicting volatility using a LASSO regression. 
LASSO stands for Least Absolute Shrinkage and Selection Operator and is just regression with L1 penalty. It is useful in stabilising parameter estimates as it produces sparse parameter estimates (exact zeros as opposed to small values). 

I have attached sample of crypto data as well. prices.csv has prices sampled every 30 minutes of 20 currencies. bid & ask.csv contain bid and ask volumes from order book snapshots at 30 min frequency and logvoldiff.csv contains log(bid/ask). Timestamps of all files should already be aligned, but it is good to check and make sure of it.

In [ ]:
# requirements.txt
# numpy
# pandas 
# pickle
# statsmodels
# matplotlib
# pandas_profiling 
# scikit-learn

##### Imports

In [1]:
# Import packages
## Currently used conda environment for local system: (defi)
import os
import sys
import math
import time
import json
import random
import requests
import numpy as np
import pandas as pd
from pickle import dump
from random import random
import statsmodels.api as sm
import matplotlib.pyplot as plt
from pandas.plotting import lag_plot
from pandas_profiling import ProfileReport
from statsmodels.tsa.ar_model import AutoReg
from pandas_visual_analysis import VisualAnalysis
from statsmodels.tsa.stattools  import   grangercausalitytests
from sklearn.metrics import mean_squared_error, mean_absolute_error
%matplotlib inline 


In [2]:
prices_df     = pd.read_csv('data/prices.csv')
logvoldiff_df = pd.read_csv('data/logvoldiff.csv')
bid_df        = pd.read_csv('data/bid.csv')
ask_df        = pd.read_csv('data/ask.csv')

In [3]:
print("Prices DF shape:     ", prices_df.shape)
print("LogVolDiff DF shape: ", logvoldiff_df.shape)
print("BID DF shape:        ", bid_df.shape)
print("ASK DF shape:        ", ask_df.shape)

Prices DF shape:      (11073, 20)
LogVolDiff DF shape:  (11073, 20)
BID DF shape:         (11073, 20)
ASK DF shape:         (11073, 20)


##### Head

In [ ]:
prices_df.head()

In [ ]:
logvoldiff_df.head()

In [ ]:
bid_df.head()

In [ ]:
ask_df.head()

##### Describe

In [ ]:
prices_df.describe()

In [ ]:
logvoldiff_df.describe()

In [ ]:
bid_df.describe()

In [ ]:
ask_df.describe()

#####  Technical Indicators Theory

1) Moving Average  (MA) - It  produces  a  buying  signal  when  the price exceeds its  most  recent  low  by  more  than  a  given  percentage  and  produces  a  selling signal when it moves in the opposite direction.  

short = 1,2,3
long  = 9,12 

(5)

2) Momentum - In  the  rule,  a  sell  signal  indicates  that value is  lower  than  its  level  of  m  periods  ago, which  implies  “negative”  momentum  and  a  low  stock  return.  We  compute the monthly signals for 1,2,3,6,9,12m.

(6)

3) Volume Data  - where k VOL is  a  measure of  the trading  volume  during  period  k.  A relatively high volume in the recent period will form a buy signal.

short = 1,2,3
long  = 9,12 

(5)


##### EDA ( Whole Data ) 

In [ ]:
prices_profile = ProfileReport(prices_df, title='prices_profile', explorative=True, dark_mode=True, progress_bar=False)
# prices_profile

In [ ]:
logvoldiff_profile = ProfileReport(logvoldiff_df, title='logvoldiff_profile', explorative=True, dark_mode=True, progress_bar=False)
# logvoldiff_profile

In [ ]:
bid_profile = ProfileReport(bid_df, title='bid_profile', explorative=True, dark_mode=True, progress_bar=False)
# bid_profile

In [ ]:
ask_profile = ProfileReport(ask_df, title='ask_profile', explorative=True, dark_mode=True, progress_bar=False)
# ask_profile

#####  Stock Return Voalitility (1inch)

In [ ]:
prices_df.columns

In [5]:
WINDOW = 240
CRYPTO = 'bal'
LAGS   = 6

In [6]:
test_df = pd.DataFrame()
test_df['date']             = prices_df['datetime']
test_df['prices']           = prices_df[CRYPTO] 
test_df['ask']              = ask_df[CRYPTO] 
test_df['bid']              = bid_df[CRYPTO] 
test_df['vol']              = 0.5 * (test_df['ask'] + test_df['bid'])
test_df['logvoldiff']       = logvoldiff_df[CRYPTO]
# test_df['Rt']               = np.log(test_df['prices'].rolling(2).mean())
# test_df['Rt2']              = test_df['Rt'] * test_df['Rt']
# test_df['RV(WINDOW)']       = test_df['Rt2'].rolling(WINDOW).mean()
# test_df['daily_price_diff'] = test_df['prices'] - test_df['prices'].shift(1)
# test_df['obv']              = test_df[['daily_price_diff']][1:].apply(lambda x: 1 if x['daily_price_diff']>0 \
#                                                                       else 0, axis=1) * test_df['Rt']

# test_df['Rt']               = np.log(test_df['prices'].rolling(2).mean())
# test_df['Rt2']              = test_df['Rt'] * test_df['Rt']

test_df['Rt']               = np.log(test_df['prices']) - np.log(test_df['prices'].shift(1))
test_df['Rt2']              = test_df['Rt'] * test_df['Rt']
test_df['RV(WINDOW)']       = test_df['Rt2'].rolling(WINDOW).mean()
test_df['daily_price_diff'] = test_df['prices'] - test_df['prices'].shift(1)
test_df['obv']              = test_df[['daily_price_diff']].apply(lambda x: 1 if x['daily_price_diff']>0 else 0, axis=1) \
                                * test_df['vol']

test_df = test_df.drop(['date'], axis=1)

In [7]:
test_df

,prices,ask,bid,vol,logvoldiff,Rt,Rt2,RV(WINDOW),daily_price_diff,obv
0,45.065,8974.2310,8360.846,8667.53850,-0.070798,NaN,NaN,NaN,NaN,0.00000
1,44.680,9431.8130,8701.479,9066.64600,-0.080595,-0.008580,0.000074,NaN,-0.385,0.00000
2,44.822,9240.2280,8863.599,9051.91350,-0.041614,0.003173,0.000010,NaN,0.142,9051.91350
3,45.296,9755.9230,8119.536,8937.72950,-0.183602,0.010520,0.000111,NaN,0.474,8937.72950
4,45.080,9523.8620,8033.560,8778.71100,-0.170173,-0.004780,0.000023,NaN,-0.216,0.00000
...,...,...,...,...,...,...,...,...,...,...
11068,22.235,17176.8520,10423.850,13800.35100,-0.499466,-0.001124,0.000001,0.000072,-0.025,0.00000
11069,21.948,14059.8990,10076.301,12068.10000,-0.333140,-0.012992,0.000169,0.000073,-0.287,0.00000
11070,22.017,14107.3100,11318.000,12712.65500,-0.220299,0.003139,0.000010,0.000072,0.069,12712.65500
11071,21.873,13884.0000,11887.360,12885.68000,-0.155261,-0.006562,0.000043,0.000072,-0.144,0.00000


##### Technical Indicators

In [8]:
# Moving Average ( s = 1,2,3), l = (9,12)
test_df['pricesma2']   = test_df['prices'].rolling(2).mean()
test_df['pricesma3']   = test_df['prices'].rolling(3).mean()
test_df['pricesma9']   = test_df['prices'].rolling(9).mean()
test_df['pricesma12']  = test_df['prices'].rolling(12).mean()

In [9]:
# Momentum Signal is computed monthly so analogoues to our use case, compunting it in windows.
# ( s = 1,2,3), l = (9,12)
test_df['pricesmo2']   = test_df['prices'].rolling(WINDOW * 2).mean()
test_df['pricesmo3']   = test_df['prices'].rolling(WINDOW * 3).mean()
test_df['pricesmo6']   = test_df['prices'].rolling(WINDOW * 6).mean()
test_df['pricesmo9']   = test_df['prices'].rolling(WINDOW * 9).mean()
test_df['pricesmo12']  = test_df['prices'].rolling(WINDOW * 12).mean()

In [10]:
#On Balance Volume ( s = 1,2,3), l = (9,12)
test_df['obv2']        = test_df['obv'].rolling(WINDOW * 2).mean()
test_df['obv3']        = test_df['obv'].rolling(WINDOW * 3).mean()
test_df['obv6']        = test_df['obv'].rolling(WINDOW * 6).mean()
test_df['obv9']        = test_df['obv'].rolling(WINDOW * 9).mean()
test_df['obv12']       = test_df['obv'].rolling(WINDOW * 12).mean()

#####  EDA ( Crypt Data)

In [ ]:
CRYPTO_profile = ProfileReport(test_df, title='profile', explorative=True, dark_mode=True, progress_bar=False)
# CRYPTO_profile

In [ ]:
# CRYPTO_profile.to_file("CRYPTO_profile.html")

In [ ]:
# gc_res = grangercausalitytests(test_df[['RV(WINDOW)', 'Rt2']], 12)

In [ ]:
gc_df = test_df[WINDOW:].copy()
gc_df = gc_df.fillna(gc_df.mean())

In [ ]:
gc_res = grangercausalitytests(gc_df[['RV(WINDOW)', 'ask']], 12)

In [ ]:
gc_res = grangercausalitytests(gc_df[['RV(WINDOW)', 'prices']], 12)

In [ ]:
gc_res = grangercausalitytests(gc_df[['RV(WINDOW)', 'obv']], 12)

In [ ]:
gc_res = grangercausalitytests(gc_df[['RV(WINDOW)', 'logvoldiff']], 12)

In [ ]:
test_df['RV(WINDOW)'].describe()

In [ ]:
test_df['RV(WINDOW)'].hist(bins=100)
mean = test_df['RV(WINDOW)'].mean()
std  = test_df['RV(WINDOW)'].std()
plt.axvline(mean,color='red',linestyle='dashed',linewidth=2)
#to plot the std line we plot both the positive and negative values 
plt.axvline(std,color='g',linestyle='dashed',linewidth=2)
plt.axvline(-std,color='g',linestyle='dashed',linewidth=2)

In [ ]:
test_df['RV(WINDOW)'].kurtosis()
#distribution is flatter than a normal curve with the same mean and standard deviation

#####  Forcasting Methods 

######  Autoregression Model

The autoregression (AR) method models the next step in the sequence as a linear function of the observations at prior time steps. Parameters of the model:

Number of AR (Auto-Regressive) terms (p): p is the parameter associated with the auto-regressive aspect of the model, which incorporates past values i.e lags of dependent variable. For instance if p is 5, the predictors for x(t) will be x(t-1)….x(t-5).

![alt text](equation.png "Title")

In [11]:
# Voalatity to be predicted
# Leaving first WINDOW variables as they have NAN
y = test_df[WINDOW:]['RV(WINDOW)'] 
# Independent variables
X = test_df[WINDOW:].drop(['RV(WINDOW)'], axis=1)
X = X.fillna(X.mean())
y = y.fillna(y.mean())
X = sm.add_constant(X) 

In [12]:
# Preparing Common Data from Autoression Model and OLS model
train_y = y[:-WINDOW]
train_X = X[:-WINDOW]
test_X  = X[-WINDOW:]
test_y  = y[-WINDOW:]
print("Training Data shape : ", train_X.shape, train_y.shape)
print("Testing  Data shape : ", test_X.shape, test_y.shape)

Training Data shape :  (10593, 24) (10593,)
Testing  Data shape :  (240, 24) (240,)


In [14]:
# Preparing autoregression data
train_ar = train_y
test_ar  = test_y.values
LAGS     = 6

# Train autoregression
model = AutoReg(train_ar,lags=LAGS)
# Fit the unconditional maximum likelihood of an AR(p) process.
model_fit = model.fit()


In [15]:
window = LAGS#model_fit.ar_lags
coef = model_fit.params

In [16]:
history = train_ar[len(train_ar)-window:].values
history = [history[i] for i in range(len(history))]
# The list of predictions
predictions = list()

In [17]:
# Walk-forward validation
# Walk forward over time steps in test
for t in range(len(test_ar)):
    length = len(history)
    lag = [history[i] for i in range(length-window, length)]
    yhat = coef[0]
    for d in range(window):
        yhat += coef[d+1] * lag[window-d-1]
    obs = test_ar[t]
    # Store forecast
    predictions.append(yhat)
    history.append(obs)

In [18]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

# Mean Squared Error – átlagos négyzetes hiba
mse = mean_squared_error(test_ar, predictions)
print('MSE:  ' + str(mse))
# Mean Absolute Error
mae = mean_absolute_error(test_ar, predictions)
print('MAE:  ' + str(mae))
# Root Mean Squared Error - a MSE négyzetgyöke
rmse = math.sqrt(mean_squared_error(test_ar, predictions))
print('RMSE: ' + str(rmse))

MSE:  5.60347931653173e-13
MAE:  4.7413018446323856e-07
RMSE: 7.485639128712878e-07


#####  OLS Estimation of the importance of other features

![alt text](eq2.png "OLS")

In [ ]:
# adding volatility variables

###### Using single lag of volatility

In [19]:
ols_df = test_df.copy()

In [21]:
y = ols_df[WINDOW:]['RV(WINDOW)'] 
# Independent variables
X = ols_df[WINDOW:].drop(['RV(WINDOW)'], axis=1)
X = X.fillna(X.mean())
y = y.fillna(y.mean())
X = sm.add_constant(X) 

# Preparing Common Data from Autoression Model and OLS model
train_y = y[:-WINDOW]
train_X = X[:-WINDOW]
test_X  = X[-WINDOW:]
test_y  = y[-WINDOW:]

In [22]:
est = sm.OLS(train_y, train_X)

In [23]:
est = est.fit()
print(est.summary())

                            OLS Regression Results                            
Dep. Variable:             RV(WINDOW)   R-squared:                       0.293
Model:                            OLS   Adj. R-squared:                  0.292
Method:                 Least Squares   F-statistic:                     199.5
Date:                Wed, 05 Jan 2022   Prob (F-statistic):               0.00
Time:                        02:46:00   Log-Likelihood:                 88587.
No. Observations:               10593   AIC:                        -1.771e+05
Df Residuals:                   10570   BIC:                        -1.770e+05
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const              4.78e-05   3.54e-06  

In [24]:
predictions = est.predict(test_X).values

In [25]:
MSE = mean_squared_error(test_y, predictions)
MAE = mean_absolute_error(test_y, predictions)

In [26]:
print("Mean Square Error :", MSE)
print("Mean Absolute Error :", MAE)

Mean Square Error : 1.6341975275465552e-09
Mean Absolute Error : 3.930277718838919e-05


In [27]:
# P values illustrates the significance of different features.
est.pvalues

const               3.778535e-41
prices              9.261213e-01
ask                 5.254267e-08
bid                 4.316690e-02
vol                 1.364284e-45
logvoldiff          1.776925e-29
Rt                  2.085560e-01
Rt2                 6.230552e-31
daily_price_diff    8.617685e-01
obv                 3.630065e-01
pricesma2           8.385785e-01
pricesma3           7.209552e-01
pricesma9           2.271050e-02
pricesma12          7.502332e-05
pricesmo2           2.962140e-61
pricesmo3           5.120650e-13
pricesmo6           2.865067e-44
pricesmo9           0.000000e+00
pricesmo12          3.884012e-73
obv2                8.359528e-03
obv3                9.919280e-07
obv6                4.412927e-09
obv9                4.551671e-27
obv12               7.772087e-16
dtype: float64

In [28]:
# These are the residuals, which we could manually add to our measured values to obtain the line of best fit
est.resid

240      0.000156
241      0.000164
242      0.000168
243      0.000168
244      0.000168
           ...   
10828   -0.000041
10829   -0.000039
10830   -0.000040
10831   -0.000040
10832   -0.000041
Length: 10593, dtype: float64

In [ ]:
# from statsmodels.stats.outliers_influence import summary_table
# st, data, ss2 = summary_table(est, alpha=0.05)

In [ ]:
# https://github.com/urschrei/linalg/blob/master/notebooks/ordinary_least_squares.ipynb

###### Using k Lags of volatility

In [29]:
ols_df = test_df[WINDOW:].copy()
y = ols_df[['RV(WINDOW)']]
y = y.reset_index()
y = y.drop(['index'], axis=1)

In [30]:
ols_df.shape

(10833, 24)

In [31]:
ind     = LAGS
columns = []


# creating k-lags columns
for _ in range(1,LAGS+1):
    columns.append('t-'+ str(_))
NEW_DATA = pd.DataFrame(columns=columns)
# filling nan for starting k lags values
for _ in range(LAGS+1):
    lag_data = []
    for __ in range(1,LAGS+1):
        lag_data.append(np.nan)
    NEW_DATA.loc[_] = lag_data
    
# appending k-previous lags values in a row
for row in y[LAGS:].iterrows():
    lag_data = []
    for lag in range(1,LAGS+1):
        lag_data.append(y.loc[row[0] - lag]['RV(WINDOW)'])
    NEW_DATA.loc[ind] = lag_data
    ind += 1

In [32]:
NEW_DATA = NEW_DATA.fillna(NEW_DATA.mean())

In [33]:
# ols_df = pd.concat([ols_df, NEW_DATA], axis=1)
for _ in range(1,LAGS+1):
    ols_df['t-'+ str(_)] = NEW_DATA['t-'+ str(_)].values

In [34]:
# y = ols_df[WINDOW:][['RV(WINDOW)']]
y = ols_df['RV(WINDOW)']
X = ols_df.drop(['RV(WINDOW)'], axis=1)
X = X.reset_index()
X = X.drop(['index'], axis=1)
X = X.fillna(X.mean())
y = y.fillna(y.mean())
# X = (X - X.mean()) / X.std()
X = sm.add_constant(X)




In [35]:
# Preparing Common Data from Autoression Model and OLS model
train_y = y[:-WINDOW]
train_X = X[:-WINDOW]
test_X  = X[-WINDOW:]
test_y  = y[-WINDOW:]
train_X = train_X.reset_index()
train_y = train_y.reset_index()
train_X = train_X.drop(['index'], axis=1)
train_y = train_y.drop(['index'], axis=1)

In [36]:
est = sm.OLS(train_y, train_X)
est = est.fit()
print(est.summary())

                            OLS Regression Results                            
Dep. Variable:             RV(WINDOW)   R-squared:                       0.986
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                 2.656e+04
Date:                Wed, 05 Jan 2022   Prob (F-statistic):               0.00
Time:                        02:46:34   Log-Likelihood:             1.0936e+05
No. Observations:               10593   AIC:                        -2.187e+05
Df Residuals:                   10564   BIC:                        -2.184e+05
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             3.999e-07   5.03e-07  

In [ ]:
# squaredbool, default=True
# If True returns MSE value, if False returns RMSE value.

In [37]:
predictions = est.predict(test_X).values
MSE         = mean_squared_error(test_y, predictions, squared=True)
RMSE        = mean_squared_error(test_y, predictions,squared=False)
MAE         = mean_absolute_error(test_y, predictions)
print("Root Mean Square Error :", RMSE)
print("Mean Square Error      :", MSE)
print("Mean Absolute Error    :", MAE)

Root Mean Square Error : 4.3448361989181585e-07
Mean Square Error      : 1.8877601595429591e-13
Mean Absolute Error    : 3.067096030073526e-07


In [ ]:
# https://github.com/jiwidi/time-series-forecasting-with-python/blob/master/03-Results_analysis%26discussion.ipynb

###### Shrinkage Approach (Lasso Regression) And Adaptive Lasso

In [ ]:
# https://machinelearningmastery.com/lasso-regression-with-python/
# https://alex.miller.im/posts/linear-model-custom-loss-function-regularization-python/

In [38]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [39]:
reg = Lasso(alpha=0.1)
reg.fit(train_X, train_y)
pred_train_lr= reg.predict(train_X)

MSE         = mean_squared_error(train_y, pred_train_lr, squared=True)
RMSE        = mean_squared_error(train_y, pred_train_lr,squared=False)
MAE         = mean_absolute_error(train_y, pred_train_lr)
print("Train")
print("Root Mean Square Error :", RMSE)
print("Mean Square Error      :", MSE)
print("Mean Absolute Error    :", MAE)
print("R2 Score               :", r2_score(train_y, pred_train_lr))

pred_test_lr= reg.predict(test_X)
MSE         = mean_squared_error(test_y, pred_test_lr, squared=True)
RMSE        = mean_squared_error(test_y, pred_test_lr,squared=False)
MAE         = mean_absolute_error(test_y, pred_test_lr)
print("Test")
print("Root Mean Square Error :", RMSE)
print("Mean Square Error      :", MSE)
print("Mean Absolute Error    :", MAE)
print("R2 Score               :", r2_score(test_y, pred_test_lr))



Train
Root Mean Square Error : 6.662357139355828e-05
Mean Square Error      : 4.438700265232558e-09
Mean Absolute Error    : 4.592094150435825e-05
R2 Score               : 0.016557149366999813
Test
Root Mean Square Error : 5.6779455280986264e-05
Mean Square Error      : 3.223906542005519e-09
Mean Absolute Error    : 5.6073957388112985e-05
R2 Score               : -39.46552330155483


In [40]:
import numpy as np
import asgl

In [42]:
train_y.values[:,0].shape

(10593,)

In [43]:
lambda1 = 10.0 ** np.arange(-3, 1.51, 0.1)

tvt_lasso = asgl.TVT(model='lm', penalization='lasso', lambda1=lambda1, parallel=True,
                     error_type='MSE', random_state=1)
lasso_result = tvt_lasso.train_validate_test(x=train_X.values, y=train_y.values[:,0])

lasso_prediction_error = lasso_result['test_error']
lasso_betas = lasso_result['optimal_betas'][1:] # Remove intercept

In [44]:
tvt_alasso = asgl.TVT(model='lm', penalization='alasso', lambda1=lambda1, parallel=True,
                      weight_technique='lasso', error_type='MSE', random_state=1, 
                      train_size=80, validate_size=20)
alasso_result = tvt_alasso.train_validate_test(x=train_X.values, y=train_y.values[:,0])

alasso_prediction_error = alasso_result['test_error']
alasso_betas = alasso_result['optimal_betas'][1:] # Remove intercept

In [45]:
print("Lasso Error          :", lasso_prediction_error)
print("Adaptive Lasso Error :", alasso_prediction_error) 

Lasso Error          : 4.396925570826056e-09
Adaptive Lasso Error : 4.636127799228995e-09


#####  Elastic Net

In [46]:
from sklearn.linear_model import ElasticNet

In [47]:
regr = ElasticNet(random_state=0)

In [48]:
regr.fit(train_X, train_y)

ElasticNet(random_state=0)

In [49]:
predictions = regr.predict(test_X)
MSE         = mean_squared_error(test_y, predictions, squared=True)
RMSE        = mean_squared_error(test_y, predictions,squared=False)
MAE         = mean_absolute_error(test_y, predictions)
print("Test")
print("Root Mean Square Error :", RMSE)
print("Mean Square Error      :", MSE)
print("Mean Absolute Error    :", MAE)
print("R2 Score               :", r2_score(test_y, predictions))


Test
Root Mean Square Error : 5.661571259623712e-05
Mean Square Error      : 3.205338912779722e-09
Mean Absolute Error    : 5.5907678010527224e-05
R2 Score               : -39.2324679001958


#### API

In [50]:
AVAILABLE_CRYPTOS = list(prices_df.columns)
AVAILABLE_CRYPTOS.remove('bake')
AVAILABLE_CRYPTOS.remove('alice')
# these cryptos have majority of Nan values

In [51]:
CRYPTO       = '1inch'
assert CRYPTO in AVAILABLE_CRYPTOS
PRICE_INPUT  = prices_df[CRYPTO]
BID_INPUT    = bid_df[CRYPTO]
ASK_INPUT    = ask_df[CRYPTO]
LOG_DIFF     = logvoldiff_df[CRYPTO]
WINDOW       = 240
LAGS         = 6

In [52]:
def preprocessing(PRICE_INPUT, BID_INPUT, ASK_INPUT, LOG_DIFF):
    test_df = pd.DataFrame()
    test_df['date']             = prices_df['datetime']
    test_df['prices']           = PRICE_INPUT
    test_df['ask']              = ASK_INPUT
    test_df['bid']              = BID_INPUT
    test_df['vol']              = 0.5 * (test_df['ask'] + test_df['bid'])
    test_df['vol']              = 0.5 * (test_df['ask'] + test_df['bid'])
    test_df['logvoldiff']       = LOG_DIFF
    test_df['Rt']               = np.log(test_df['prices']) - np.log(test_df['prices'].shift(1))
    test_df['Rt2']              = test_df['Rt'] * test_df['Rt']
    test_df['RV(WINDOW)']       = test_df['Rt2'].rolling(WINDOW).mean()
    test_df['daily_price_diff'] = test_df['prices'] - test_df['prices'].shift(1)
    test_df['obv']              = test_df[['daily_price_diff']].apply(lambda x: 1 if x['daily_price_diff']>0 else 0, axis=1) \
                                    * test_df['vol']

    test_df = test_df.drop(['date'], axis=1)
    
    # Moving Average ( s = 1,2,3), l = (9,12)
    test_df['pricesma2']   = test_df['prices'].rolling(2).mean()
    test_df['pricesma3']   = test_df['prices'].rolling(3).mean()
    test_df['pricesma9']   = test_df['prices'].rolling(9).mean()
    test_df['pricesma12']   = test_df['prices'].rolling(12).mean()
    # Momentum Signal is computed monthly so analogoues to our use case, compunting it in windows.
    # ( s = 1,2,3), l = (9,12)
    test_df['pricesmo2']   = test_df['prices'].rolling(WINDOW * 2).mean()
    test_df['pricesmo3']   = test_df['prices'].rolling(WINDOW * 3).mean()
    test_df['pricesmo6']   = test_df['prices'].rolling(WINDOW * 6).mean()
    test_df['pricesmo9']   = test_df['prices'].rolling(WINDOW * 9).mean()
    test_df['pricesmo12']   = test_df['prices'].rolling(WINDOW * 12).mean()
    #On Balance Volume ( s = 1,2,3), l = (9,12)
    test_df['obv2']        = test_df['obv'].rolling(WINDOW * 2).mean()
    test_df['obv3']        = test_df['obv'].rolling(WINDOW * 3).mean()
    test_df['obv6']        = test_df['obv'].rolling(WINDOW * 6).mean()
    test_df['obv9']        = test_df['obv'].rolling(WINDOW * 9).mean()
    test_df['obv12']        = test_df['obv'].rolling(WINDOW * 12).mean()
    
    return test_df

In [53]:
def train_test_split(test_df): 
    # Voalatity to be predicted
    # Leaving first WINDOW variables as they have NAN
    y = test_df[WINDOW:]['RV(WINDOW)'] 
    # Independent variables
    X = test_df[WINDOW:].drop(['RV(WINDOW)'], axis=1)
    X = X.fillna(X.mean())
    y = y.fillna(y.mean())
    X = sm.add_constant(X) 

    # Preparing Common Data from Autoression Model and OLS model
    train_y = y[:-WINDOW]
    train_X = X[:-WINDOW]
    test_X  = X[-WINDOW:]
    test_y  = y[-WINDOW:]
    print("Training Data shape : ", train_X.shape, train_y.shape)
    print("Testing  Data shape : ", test_X.shape, test_y.shape)
    
    return train_y, train_X,test_X ,test_y

In [54]:
def AR_MODEL(train_ar,test_ar, LAGS, save_model = False, return_model = False):
    model     = AutoReg(train_ar,lags=LAGS)
    model_fit = model.fit()
    window    = LAGS
    coef      = model_fit.params
    history   = train_ar[len(train_ar)-window:]
    history   = [history[i] for i in range(len(history))]
    # The list of predictions
    predictions = list()

    # Walk-forward validation
    # Walk forward over time steps in test
    for t in range(len(test_ar)):
        length = len(history)
        lag = [history[i] for i in range(length-window, length)]
        yhat = coef[0]
        for d in range(window):
            yhat += coef[d+1] * lag[window-d-1]
        obs = test_ar[t]
        # Store forecast
        predictions.append(yhat)
        history.append(obs)
    
        # Mean Squared Error – átlagos négyzetes hiba
    mse = mean_squared_error(test_ar, predictions)
    print('MSE:  ' + str(mse))
    # Mean Absolute Error
    mae = mean_absolute_error(test_ar, predictions)
    print('MAE:  ' + str(mae))
    # Root Mean Squared Error - a MSE négyzetgyöke
    rmse = math.sqrt(mean_squared_error(test_ar, predictions))
    print('RMSE: ' + str(rmse))
    
    if save_model:
        dump(model, open('AR_MODEL.pkl', 'wb'))
    
    if return_model:
        return model     

In [56]:
exp_df = preprocessing(PRICE_INPUT, BID_INPUT, ASK_INPUT, LOG_DIFF)

In [57]:
train_y, train_X,test_X ,test_y = train_test_split(exp_df)

Training Data shape :  (10593, 24) (10593,)
Testing  Data shape :  (240, 24) (240,)


In [58]:
AR_MODEL(train_y.values, test_y.values, LAGS)

MSE:  7.872663163689955e-13
MAE:  5.420830236296093e-07
RMSE: 8.872802918858254e-07


In [61]:
def OLS_1_LAG(ols_df, save_model = False, return_model = False):
    y = ols_df[WINDOW:]['RV(WINDOW)'] 
    # Independent variables
    X = ols_df[WINDOW:].drop(['RV(WINDOW)'], axis=1)
    X = X.fillna(X.mean())
    y = y.fillna(y.mean())
    X = sm.add_constant(X) 

    # Preparing Common Data from Autoression Model and OLS model
    train_y = y[:-WINDOW]
    train_X = X[:-WINDOW]
    test_X  = X[-WINDOW:]
    test_y  = y[-WINDOW:]
    
    print(y.shape)
    
    est = sm.OLS(train_y, train_X)
    
    est = est.fit()
    print(est.summary())
    
    predictions = est.predict(test_X).values
    MSE  = mean_squared_error(test_y, predictions, squared=True)
    RMSE = mean_squared_error(test_y, predictions, squared=False)
    MAE  = mean_absolute_error(test_y, predictions)
    print('\n')
    print("Root Mean Square Error :", RMSE)
    print("Mean Square Error      :", MSE)
    print("Mean Absolute Error    :", MAE)
    
    
    if save_model:
        dump(model, open('OLS_k_Lags.pkl', 'wb'))
    if return_model:
        return model    

In [62]:
OLS_1_LAG(exp_df)

(10833,)
                            OLS Regression Results                            
Dep. Variable:             RV(WINDOW)   R-squared:                       0.158
Model:                            OLS   Adj. R-squared:                  0.156
Method:                 Least Squares   F-statistic:                     90.27
Date:                Wed, 05 Jan 2022   Prob (F-statistic):               0.00
Time:                        02:50:17   Log-Likelihood:                 81633.
No. Observations:               10593   AIC:                        -1.632e+05
Df Residuals:                   10570   BIC:                        -1.631e+05
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0001   1

In [ ]:
# def OLS(ols_df):
    
#     y = ols_df[WINDOW:]['RV(WINDOW)'] 
#     y = y.fillna(y.mean())
#     y = y.reset_index()
#     y = y.drop(['index'], axis=1)
    
#     ind     = 0
#     columns = []
#     for _ in range(LAGS):
#         columns.append('t-'+ str(_))
#     NEW_DATA = pd.DataFrame(columns=columns)
#     for row in y[LAGS:].iterrows():
#         lag_data = []
#         for lag in range(LAGS):
#             lag_data.append(y.loc[row[0] - lag]['RV(WINDOW)'])
#         NEW_DATA.loc[ind] = lag_data
#         ind += 1
        
#     ols_df = pd.concat([ols_df, NEW_DATA], axis=1)
    
    
#     # Independent variables
#     X = ols_df[WINDOW:].drop(['RV(WINDOW)', 'date'], axis=1)
#     X = X.fillna(X.mean())
    
# #     X = sm.add_constant(X) 

#     # Preparing Common Data from Autoression Model and OLS model
#     train_y = y[:-WINDOW]
#     train_X = X[:-WINDOW]
#     test_X  = X[-WINDOW:]
#     test_y  = y[-WINDOW:]
#     train_X = train_X.reset_index()
#     train_y = train_y.reset_index()
#     train_X = train_X.drop(['index'], axis=1)
#     train_y = train_y.drop(['index'], axis=1)
    
#     est = sm.OLS(train_y, train_X)
    
#     est = est.fit()
#     print(est.summary())
    
    
#     predictions = est.predict(test_X).values
#     MSE  = mean_squared_error(test_y, predictions, squared=True)
#     RMSE = mean_squared_error(test_y, predictions,squared=False)
#     MAE  = mean_absolute_error(test_y, predictions)
    
#     print('\n')
#     print("Root Mean Square Error :", RMSE)
#     print("Mean Square Error      :", MSE)
#     print("Mean Absolute Error    :", MAE)
# OLS(exp_df)

In [63]:
def OLS(test_df, save_model = False, return_model = False):
    ols_df = test_df[WINDOW:].copy()
    y = ols_df[['RV(WINDOW)']]
    y = y.reset_index()
    y = y.drop(['index'], axis=1)
    
    ind     = LAGS
    columns = []

    # creating k-lags columns
    for _ in range(1,LAGS+1):
        columns.append('t-'+ str(_))
    NEW_DATA = pd.DataFrame(columns=columns)
    # filling nan for starting k lags values
    for _ in range(LAGS+1):
        lag_data = []
        for __ in range(1,LAGS+1):
            lag_data.append(np.nan)
        NEW_DATA.loc[_] = lag_data

    # appending k-previous lags values in a row
    for row in y[LAGS:].iterrows():
        lag_data = []
        for lag in range(1,LAGS+1):
            lag_data.append(y.loc[row[0] - lag]['RV(WINDOW)'])
        NEW_DATA.loc[ind] = lag_data
        ind += 1
        
    NEW_DATA = NEW_DATA.fillna(NEW_DATA.mean())
    
    # ols_df = pd.concat([ols_df, NEW_DATA], axis=1)
    for _ in range(1,LAGS+1):
        ols_df['t-'+ str(_)] = NEW_DATA['t-'+ str(_)].values

        # y = ols_df[WINDOW:][['RV(WINDOW)']]
    y = ols_df['RV(WINDOW)']
    X = ols_df.drop(['RV(WINDOW)'], axis=1)
    X = X.reset_index()
    X = X.drop(['index'], axis=1)
    X = X.fillna(X.mean())
    y = y.fillna(y.mean())
    X = sm.add_constant(X) 
    
    # Preparing Common Data from Autoression Model and OLS model
    train_y = y[:-WINDOW]
    train_X = X[:-WINDOW]
    test_X  = X[-WINDOW:]
    test_y  = y[-WINDOW:]
    train_X = train_X.reset_index()
    train_y = train_y.reset_index()
    train_X = train_X.drop(['index'], axis=1)
    train_y = train_y.drop(['index'], axis=1)
    est = sm.OLS(train_y, train_X)
    est = est.fit()
    print(est.summary())
    
    predictions = est.predict(test_X).values
    MSE  = mean_squared_error(test_y, predictions, squared=True)
    RMSE = mean_squared_error(test_y, predictions,squared=False)
    MAE  = mean_absolute_error(test_y, predictions)
    print('\n')
    print("Root Mean Square Error :", RMSE)
    print("Mean Square Error      :", MSE)
    print("Mean Absolute Error    :", MAE)
    
    
    if save_model:
        dump(model, open('OLS_k_Lags.pkl', 'wb'))
    if return_model:
        return model     

In [64]:
OLS(exp_df)

                            OLS Regression Results                            
Dep. Variable:             RV(WINDOW)   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                 3.278e+04
Date:                Wed, 05 Jan 2022   Prob (F-statistic):               0.00
Time:                        02:50:44   Log-Likelihood:             1.0443e+05
No. Observations:               10593   AIC:                        -2.088e+05
Df Residuals:                   10564   BIC:                        -2.086e+05
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             1.326e-07   2.16e-06  